In [7]:
# =============================================================================
# Qiskit Simulation of the BB84 QKD Protocol
# This script is designed to run locally on a system with Qiskit 1.0+ installed.
# It demonstrates the core quantum mechanics of the protocol described in the
# [cite_start]"Conceptual Framework and Simulation of QKD for Secure EHRs" project[cite: 2].
# =============================================================================

# --- Step 1: Import necessary libraries ---
# --- Step 1: Import necessary libraries ---
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer  # <-- CORRECTED IMPORT
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

print("Qiskit is ready. Starting simulation...")

# --- Step 2: Define the core simulation function ---

def simulate_bb84_qubit(alice_bit, alice_basis, bob_basis, eve_is_present=False, eve_basis=0):
    """
    Simulates a single qubit exchange in the BB84 protocol using the modern Qiskit 1.0+ API.
    - Basis 0: Rectilinear (+)
    - Basis 1: Diagonal (x)
    """
    
    # === ALICE PREPARES THE QUBIT ===
    # A quantum circuit with one quantum bit (for the qubit) and one classical bit (for the measurement result).
    qc = QuantumCircuit(1, 1)
    
    # Alice encodes her classical bit (0 or 1).
    if alice_bit == 1:
        qc.x(0)  # Apply an X-gate (a bit-flip) to change the state from |0> to |1>.
    
    # Alice chooses her basis.
    if alice_basis == 1:
        qc.h(0)  # Apply a Hadamard gate to switch to the diagonal basis (creating superposition).
    
    qc.barrier() # A visual separator in the circuit diagram for clarity.

    # Get the simulator backend which will run our quantum circuit.
    backend = Aer.get_backend('qasm_simulator')

    # === EVE THE EAVESDROPPER INTERCEPTS (if present) ===
    if eve_is_present:
        # Eve measures the qubit using her own randomly chosen basis.
        if eve_basis == 1:
            qc.h(0)
        qc.measure(0, 0)
        
        # We run the circuit to find out what Eve measured.
        transpiled_eve_qc = transpile(qc, backend)
        job_eve = backend.run(transpiled_eve_qc, shots=1, memory=True)
        result_eve = job_eve.result()
        eve_measurement = int(result_eve.get_memory()[0])
        
        # Eve prepares a NEW qubit based on her measurement to send to Bob.
        # This is the crucial step where she introduces errors if her basis was wrong.
        qc = QuantumCircuit(1, 1) # A fresh circuit for the re-transmitted qubit.
        if eve_measurement == 1:
            qc.x(0)
        if eve_basis == 1:
            qc.h(0)
        qc.barrier()

    # === BOB MEASURES THE QUBIT ===
    # Bob applies his basis gate before measuring.
    if bob_basis == 1:
        qc.h(0)
    qc.measure(0, 0)
    
    # --- Run the final circuit on the simulator ---
    transpiled_bob_qc = transpile(qc, backend)
    job_bob = backend.run(transpiled_bob_qc, shots=1024) # Run 1024 times to get statistics
    result = job_bob.result()
    counts = result.get_counts()
    
    # Find the most likely outcome.
    measured_bit = int(max(counts, key=counts.get))
    
    return qc, counts, measured_bit

# --- Step 3: Run the full demonstration ---

if __name__ == '__main__':
    # Define a short, clear message for Alice to send.
    alice_secret_bits  = [1, 0, 1, 0]
    alice_secret_bases = [0, 1, 1, 0] # 0 for +, 1 for x

    print("="*60)
    print("DEMONSTRATION 1: SECURE KEY TRANSFER (NO EAVESDROPPER)")
    print("="*60)

    for i in range(len(alice_secret_bits)):
        alice_bit = alice_secret_bits[i]
        alice_basis = alice_secret_bases[i]
        
        # SCENARIO A: Bob chooses the SAME basis as Alice.
        bob_matching_basis = alice_basis
        _, _, measured_bit = simulate_bb84_qubit(alice_bit, alice_basis, bob_matching_basis)
        print(f"\nAlice sends bit '{alice_bit}' in basis {alice_basis}. Bob measures in matching basis {bob_matching_basis}.")
        print(f"  -> Bob's result is certain: '{measured_bit}'. Bases Match: KEEP BIT.")
        
        # SCENARIO B: Bob chooses the WRONG basis.
        bob_wrong_basis = 1 - alice_basis
        _, _, measured_bit = simulate_bb84_qubit(alice_bit, alice_basis, bob_wrong_basis)
        print(f"Alice sends bit '{alice_bit}' in basis {alice_basis}. Bob measures in wrong basis {bob_wrong_basis}.")
        print(f"  -> Bob's result is random (50/50). He gets '{measured_bit}'. Bases Mismatch: DISCARD BIT.")


    print("\n\n" + "="*60)
    print("DEMONSTRATION 2: ATTACKER IS TRACKED")
    print("="*60)

    # We'll focus on one specific exchange to show Eve's effect clearly.
    # Alice sends bit '1' in the diagonal 'x' basis (1). Bob will also measure in basis '1'.
    # Without Eve, Bob should get '1' with 100% certainty.

    alice_bit = 1
    alice_basis = 1
    bob_basis = 1

    print(f"\nAlice sends bit '{alice_bit}' in basis {alice_basis}. Bob measures in the same basis {bob_basis}.")
    print("EXPECTED RESULT WITHOUT EVE: Bob should always measure '1'.")

    # SCENARIO C: Eve intercepts but guesses the basis CORRECTLY.
    eve_correct_basis = 1
    _, _, measured_bit = simulate_bb84_qubit(alice_bit, alice_basis, bob_basis, eve_is_present=True, eve_basis=eve_correct_basis)
    print(f"\nEve intercepts and correctly guesses basis {eve_correct_basis}.")
    print(f"  -> Bob's result: '{measured_bit}'. Eve remains HIDDEN in this instance.")

    # SCENARIO D: Eve intercepts and guesses the basis INCORRECTLY.
    eve_wrong_basis = 0
    final_qc, final_counts, measured_bit = simulate_bb84_qubit(alice_bit, alice_basis, bob_basis, eve_is_present=True, eve_basis=eve_wrong_basis)
    print(f"\nEve intercepts and incorrectly guesses basis {eve_wrong_basis}.")
    print(f"  -> Bob's result: '{measured_bit}'. An ERROR has been introduced!")
    print("  -> This error would increase the Quantum Bit Error Rate (QBER), revealing the attacker's presence[cite: 33].")

    # --- Step 4: Visualize the final, most interesting result ---
    print("\nSimulation complete. Displaying plots for the final scenario...")

    # Plot the circuit diagram for the case where Eve introduces an error.
    final_qc.draw('mpl', filename='qkd_circuit.png')
    print("Saved circuit diagram as 'qkd_circuit.png'")

    # Plot the histogram of measurement results for that same case.
    plot_histogram(final_counts, title="Bob's Measurement Results When Eve Guesses Wrong")
    print("Displaying plots...")
    plt.show() # This command opens the plot windows.

Qiskit is ready. Starting simulation...
DEMONSTRATION 1: SECURE KEY TRANSFER (NO EAVESDROPPER)

Alice sends bit '1' in basis 0. Bob measures in matching basis 0.
  -> Bob's result is certain: '1'. Bases Match: KEEP BIT.
Alice sends bit '1' in basis 0. Bob measures in wrong basis 1.
  -> Bob's result is random (50/50). He gets '0'. Bases Mismatch: DISCARD BIT.

Alice sends bit '0' in basis 1. Bob measures in matching basis 1.
  -> Bob's result is certain: '0'. Bases Match: KEEP BIT.
Alice sends bit '0' in basis 1. Bob measures in wrong basis 0.
  -> Bob's result is random (50/50). He gets '0'. Bases Mismatch: DISCARD BIT.

Alice sends bit '1' in basis 1. Bob measures in matching basis 1.
  -> Bob's result is certain: '1'. Bases Match: KEEP BIT.
Alice sends bit '1' in basis 1. Bob measures in wrong basis 0.
  -> Bob's result is random (50/50). He gets '0'. Bases Mismatch: DISCARD BIT.

Alice sends bit '0' in basis 0. Bob measures in matching basis 0.
  -> Bob's result is certain: '0'. Ba

In [9]:
# =============================================================================
# Qiskit Simulation: Demonstrating Eavesdropper Detection in the BB84 Protocol
#
# This simulation provides a practical proof-of-concept for the security
# principles outlined in the "Conceptual Framework and Simulation of QKD for
# Secure Electronic Health Records (EHRs)" project report. It visually
# demonstrates how any eavesdropping leaves a "detectable signature" 
# by increasing the Quantum Bit Error Rate (QBER).
# =============================================================================

# --- Step 1: Import necessary libraries ---
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import time

print("Qiskit is ready. Starting attacker simulation...")

# --- Step 2: Define the core BB84 simulation function ---

def simulate_bb84_qubit(alice_bit, alice_basis, bob_basis, eve_is_present=False, eve_basis=0):
    """
    Simulates a single qubit exchange in the BB84 protocol [cite: 78] using the modern Qiskit API.
    """
    
    # Alice prepares a qubit based on her bit and basis choice.
    qc = QuantumCircuit(1, 1)
    if alice_bit == 1: qc.x(0)
    if alice_basis == 1: qc.h(0)
    qc.barrier()

    # Get the simulator backend.
    backend = Aer.get_backend('qasm_simulator')

    # An eavesdropper ('Eve') intercepts the qubit.
    if eve_is_present:
        # Eve measures the qubit, which is the act of eavesdropping.
        if eve_basis == 1: qc.h(0)
        qc.measure(0, 0)
        
        # This measurement collapses the quantum state.
        transpiled_eve_qc = transpile(qc, backend)
        job_eve = backend.run(transpiled_eve_qc, shots=1, memory=True)
        result_eve = job_eve.result()
        eve_measurement = int(result_eve.get_memory()[0])
        
        # Eve prepares a NEW qubit based on her measurement to send to Bob.
        # This is where the "detectable signature"  is created if her basis was wrong.
        qc = QuantumCircuit(1, 1)
        if eve_measurement == 1: qc.x(0)
        if eve_basis == 1: qc.h(0)
        qc.barrier()

    # Bob measures the qubit he receives from Eve (or Alice if no Eve).
    if bob_basis == 1:
        qc.h(0)
    qc.measure(0, 0)
    
    # Run the final circuit to see Bob's result.
    transpiled_bob_qc = transpile(qc, backend)
    job_bob = backend.run(transpiled_bob_qc, shots=1024)
    result = job_bob.result()
    counts = result.get_counts()
    
    measured_bit = int(max(counts, key=counts.get))
    return qc, counts, measured_bit

# --- Step 3: Run the Demonstration ---

if __name__ == '__main__':
    print("="*70)
    print("DEMONSTRATION: How the BB84 Protocol Detects an Eavesdropper")
    print("="*70)

    # We will simulate one specific exchange where the detection is clear.
    # Alice sends bit '1' in the diagonal 'x' basis (1).
    # Bob will also measure in the diagonal 'x' basis (1).
    # According to the protocol, without an attacker, Bob should ALWAYS get '1'.
    alice_bit = 1
    alice_basis = 1 # Diagonal 'x' basis
    bob_basis = 1   # Diagonal 'x' basis

    print(f"SETUP: Alice will send bit '{alice_bit}' using basis {alice_basis}.")
    print(f"       Bob will measure using the SAME basis {bob_basis}.")
    print("EXPECTED RESULT (No Attacker): Bob should measure '1' with 100% certainty.")
    time.sleep(2)

    # SCENARIO A: Eve intercepts but guesses the basis INCORRECTLY.
    # This is the crucial scenario that demonstrates the security.
    eve_wrong_basis = 0 # Rectilinear '+' basis
    
    print(f"\nACTION: An attacker (Eve) intercepts the qubit.")
    print(f"        Eve incorrectly guesses the measurement basis is {eve_wrong_basis} (+).")
    time.sleep(2)

    final_qc, final_counts, measured_bit = simulate_bb84_qubit(
        alice_bit, 
        alice_basis, 
        bob_basis, 
        eve_is_present=True, 
        eve_basis=eve_wrong_basis
    )
    
    print("\n--- RESULTS ---")
    print(f"Bob's actual measurement result is now '{measured_bit}'!")
    time.sleep(1)
    
    print("\n--- ANALYSIS ACCORDING TO THE PROJECT REPORT ---")
    print("The attacker's measurement introduced a detectable error.")
    print("This is the 'detectable signature'  that the QKD protocol guarantees.")
    print("In our large-scale network simulation, thousands of these errors would cause the")
    print("Quantum Bit Error Rate (QBER)  to spike, immediately signaling an attack.")
    print("This confirms that the protocol provides a provably secure channel for key exchange[cite: 28].")

    # --- Step 4: Visualize the Result ---
    print("\nVisualizing the results for the attacked scenario...")

    # Plot the circuit diagram
    final_qc.draw('mpl', filename='qkd_attack_circuit.png')
    print("Saved circuit diagram as 'qkd_attack_circuit.png'")

    # Plot the histogram of measurement results
    plot_histogram(final_counts, title="Bob's Results After Eve's Interference")
    print("Displaying plots...")
    plt.show() # This command opens the plot windows.

Qiskit is ready. Starting attacker simulation...
DEMONSTRATION: How the BB84 Protocol Detects an Eavesdropper
SETUP: Alice will send bit '1' using basis 1.
       Bob will measure using the SAME basis 1.
EXPECTED RESULT (No Attacker): Bob should measure '1' with 100% certainty.

ACTION: An attacker (Eve) intercepts the qubit.
        Eve incorrectly guesses the measurement basis is 0 (+).

--- RESULTS ---
Bob's actual measurement result is now '0'!

--- ANALYSIS ACCORDING TO THE PROJECT REPORT ---
The attacker's measurement introduced a detectable error.
This is the 'detectable signature'  that the QKD protocol guarantees.
In our large-scale network simulation, thousands of these errors would cause the
Quantum Bit Error Rate (QBER)  to spike, immediately signaling an attack.
This confirms that the protocol provides a provably secure channel for key exchange[cite: 28].

Visualizing the results for the attacked scenario...
Saved circuit diagram as 'qkd_attack_circuit.png'
Displaying plot